# Home work "CTC-loss"

Задача: реализовать прямой (forward) проход и вчислить матрицу $\alpha_t(s)$. Значения этой матрицы должны равняться
- $\alpha_t(s)$ в случае если валидные пути проходят через данную ячейку
- 0.0 в противном случае

Размерности входов и результата см. по коду.

Ноутбук с решением присыслать на voropaev@corp.mail.ru Тему письма пишите пожалуйста в формате "[МИФИ] Фамилия"

Deadline: 21.12.2018

In [2]:
import torch

In [10]:
# Список меток символов строки, для которой рассчитываем loss. Значение 0 зарезервированно для пустого символа.
l = torch.tensor([1,2,2,3,1,4,5,4,3,2,1,], dtype=torch.long)

# y[t, s] - предсказанные сетью вероятности для каждого фрейма.
y = torch.tensor(
[[1,  0.,  0., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [1., 0., 0., 0., 0., 0.],
 [0,  0., 1., 0., 0., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 1., 0., 0., 0., 0.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 0., 0., 1.],
 [0., 0., 0., 0., 1., 0.],
 [0., 0., 0., 1., 0., 0.],
 [0., 0., 1., 0., 0., 0.],
 [0., 1., 0., 0., 0., 0.],], dtype=torch.float32)
print(len(y[:]))
tensor = torch.ones(())
x = tensor.new_full((14,6), 1)
print(x)

14
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])


In [11]:
# def compute_ctc_alpha(l, y):
#     """
#     Функция, вычисляющая матрицу $\alpha$ для данного входа.
#     @param l метки символов строки, размерностью [L,]
#     @param y предсказанные сетью вероятности для каждого фрейма. Размерность [T, Lexicon+1]
#     @return матрицу $\alpha$ размерностью [2*L+1, T]
#     """
    
#     #TODO Вписать решение сюда. 
#     s = 0
#     alpha = torch.zeros((2*l.size()[0]+1, y.size()[0]))
#     word = torch.zeros((l.size()[0]*2+1))
#     for i in range(word.size()[0]):
#         if i %2 == 0:
#             word[i] = 0
#         else:
#             word[i] = l[i//2]
#     for j in range(len(alpha[0])):
#         for k in range(len(y[0])):
#             if y[j][k] != 0:
#                 for i in range (s, len(alpha[:])):
#                     if word[i] == k:
#                         alpha[i][j] = y[j][k]
#                         s = i
#                         break

# #     for j in range(1, len(alpha[0])):
# #         for i in range(len(alpha[:])):
# #             if j%2 == 1 and :
# #                 alpha[i,j] = (y[i-1,j-1] + y[i,j-1])* y[i,j]
    
#     print(alpha)
#     return alpha

In [12]:
def get_mask(l,y, word):
    r = abs(len(l)+1 - y.size()[0])
    mask = torch.zeros((2*l.size()[0]+1, y.size()[0]))
    j = 0
    i = 0
    while i<mask.size()[0] and j<mask.size()[1]:           
            if word[i] == 0 and word[i-1] == word[(i+1)%len(word)]:
                mask[i][j:j+r+1] = 1
                j+=1
            else:
                mask[i][j:j+r] = 1
            if word[i]!= 0:
                mask[i][j:j+r+1] = 1
                j+=1 
            i+=1
            
    return mask         
    
  

def compute_ctc_alpha(l, y):
    """
    Функция, вычисляющая матрицу $\alpha$ для данного входа.
    @param l метки символов строки, размерностью [L,]
    @param y предсказанные сетью вероятности для каждого фрейма. Размерность [T, Lexicon+1]
    @return матрицу $\alpha$ размерностью [2*L+1, T]
    """
    
    #TODO Вписать решение сюда. 
    s = 0
    alpha = torch.zeros((2*l.size()[0]+1, y.size()[0]))
    word = torch.zeros((l.size()[0]*2+1))
    for i in range(word.size()[0]):
        if i%2 == 0:
            word[i] = 0
        else:
            word[i] = l[i//2]
#     print(get_mask(l,y,word)) 
    mask = get_mask(l,y,word)
    for j in range(len(alpha[0])):
        for k in range(len(y[0])):
            if y[j][k] != 0:
                for i in range (s, len(alpha[:])):
                    if word[i] == k:
                        alpha[i][j] = y[j][k]
#                         s = i
                        
#     print(alpha)
    alpha *= mask

    for j in range(1,len(alpha[0])):
        for i in range(1,len(alpha[:])):
            if word[i] == 0 or word[i] == word[i-2]:
                alpha[i][j] = (alpha[i-1,j-1] + alpha[i,j-1])* alpha[i,j]
            else:
                alpha[i][j] = (alpha[i-2,j-1]+alpha[i-1,j-1] + alpha[i,j-1])* alpha[i,j]
    
    
    return alpha
compute_ctc_alpha(l,y)

tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0

In [13]:
## Этот блок приведен исключительно для примера. Реальный тест я подставлю сам. 
# Обязательно сохраните сигнатуру функции compute_ctc_alpha


def test():
    al = compute_ctc_alpha(l, y)
    ritght_al = torch.tensor([
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


    if torch.all(al == ritght_al):
        return True
    else:
        return False
    
compute_ctc_alpha(l, x)     
assert test(), "Test faled"
   